In [105]:
#!pip install pandas==2.2.2
#!pip install numpy==1.26.4
#!pip install scikit-learn==1.4.2
#!pip install joblib==1.4.2
#!pip install streamlit==1.34.0
#!pip install  --upgrade scikit-learn
#import sklearn
#sklearn.__version__

In [106]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format
df_0.head(20)

,manufacturer,model,description,price
0,NISSAN,2024,NISSAN SENTRA,"401,900"
1,NISSAN,2024,NISSAN PATHFINDER,"1,243,900"
2,NISSAN,2024,NISSAN X-TRAIL,"651,900"
3,NISSAN,2024,NISSAN VERSA,"339,900"
4,NISSAN,2024,NISSAN ALTIMA,"750,900"
5,CHEVROLET,2024,CHEVROLET AVEO,"280,400"
6,CHEVROLET,2024,CHEVROLET ONIX,"295,400"
7,CHEVROLET,2024,CHEVROLET CAVALIER,"385,900"
8,CHEVROLET,2024,CHEVROLET GROOVE,"375,900"
9,CHEVROLET,2024,CHEVROLET TRAX,"507,100"


In [108]:
df = pd.DataFrame({'manufacturer':[], 'model':[], 'description':[], 'price':[]})

for i in range(1,5000):
    random_index = np.random.choice(df_0.index)
    random_data = df_0.loc[random_index]
    df_aux = pd.DataFrame({'manufacturer':[random_data['manufacturer']], 'model':[random_data['model']], 'description':[random_data['description']], 'price':[random_data['price']]})
    df = pd.concat([df, df_aux])

random = []
for i in range(0, len(df)):
    random.append(1 + np.random.uniform(-0.2,0.2))

df['random'] = random
df['mileage'] = (2024 - df['model'] ) * 20_000 * df['random'] 
df.drop(['random'], axis = 1, inplace = True)
df

,manufacturer,model,description,price,mileage
0,NISSAN,"2,020",NISSAN ALTIMA,"437,925","77,990"
0,HONDA,"2,016",HONDA ACCORD,"289,236","183,758"
0,CHEVROLET,"2,020",CHEVROLET TRAX,"295,741","71,920"
0,HONDA,"2,023",HONDA ACCORD,"604,720","18,096"
0,CHEVROLET,"2,024",CHEVROLET ONIX,"295,400",0
...,...,...,...,...,...
0,VOLKSWAGEN,"2,024",VOLKSWAGEN NIVUS,"479,070",0
0,CHEVROLET,"2,017",CHEVROLET GROOVE,"159,815","117,397"
0,VOLKSWAGEN,"2,021",VOLKSWAGEN T-CROSS,"270,780","65,478"
0,TOYOTA,"2,020",TOYOTA RAV4,"334,057","91,858"


In [109]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical columns
categorical_cols = ['manufacturer','description']
numeric_cols = ['model', 'mileage']

# Create a pipeline for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'
)

# Create a pipeline for the model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Training R^2 score:", train_score)
print("Testing R^2 score:", test_score)


y_train_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
print("Train RMSE:", rmse)

y_test_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
print("Test RMSE:", rmse)

Training R^2 score: 0.9999570257293093
Testing R^2 score: 0.9998794591838067
Train RMSE: 1105.9059109006591
Test RMSE: 1922.5790455198444


In [110]:
df['prediction'] = model.predict(X)
df

,manufacturer,model,description,price,mileage,prediction
0,NISSAN,"2,020",NISSAN ALTIMA,"437,925","77,990","437,925"
0,HONDA,"2,016",HONDA ACCORD,"289,236","183,758","289,236"
0,CHEVROLET,"2,020",CHEVROLET TRAX,"295,741","71,920","295,741"
0,HONDA,"2,023",HONDA ACCORD,"604,720","18,096","604,720"
0,CHEVROLET,"2,024",CHEVROLET ONIX,"295,400",0,"295,400"
...,...,...,...,...,...,...
0,VOLKSWAGEN,"2,024",VOLKSWAGEN NIVUS,"479,070",0,"479,070"
0,CHEVROLET,"2,017",CHEVROLET GROOVE,"159,815","117,397","159,993"
0,VOLKSWAGEN,"2,021",VOLKSWAGEN T-CROSS,"270,780","65,478","270,780"
0,TOYOTA,"2,020",TOYOTA RAV4,"334,057","91,858","334,057"


In [111]:
# Obtener la importancia de las características
onehot_columns = list(model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_cols))
#onehot_columns
numeric_columns = numeric_cols
feature_names = onehot_columns + numeric_columns
feature_importance = model.named_steps['regressor'].feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

feature_importance_df['Importance'] = feature_importance_df['Importance'] * 1.0
#feature_importance_df

In [112]:
import pickle
filename = 'car_price_model_pickle.sav'
pickle.dump(model, open(filename, 'wb'))